## Description
Dataset contains nearly 32K tweets which are labeled based on having racist or sexist content. We are going to analyse this dataset and tweets, and by the end, create a classification model to classify tweets.   
Each row in the dataset has 3 columns:
* `id`: Assigned ID to this tweet by Analytics Vidhya.
* `label`: Tweet label, 1 if tweet has hatred content and 0 otherwise.
* `tweet`: Tweet text.  

Dataset is provided by [Analytics Vidhya](http://https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/)  

In [ ]:
import numpy as np
import pandas as pd
#import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import os
import sys
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
%matplotlib inline



In [ ]:
data = pd.read_excel('../input/germny-wteet/germeval2018-3.xlsx')
data1 = pd.read_csv('../input/multi-language-dataset/cleaned_data_hatespeech.csv')
data2 = pd.read_csv('../input/tweet-english/train_E6oV3lV.csv')
data3 = pd.read_csv('../input/french-tweet/french_tweets-003.csv')
Ru_data= pd.read_csv('../input/russian-language-toxic-comments/labeled.csv')
Ko_data= pd.read_csv('../input/korean-dataset/hate_speech_binary_dataset.csv')


In [ ]:
#Greek_data= pd.read_csv('../input/greekkkk/offenseval-gr-training-v1.tsv')


In [ ]:

Ru_data.columns =['tweet','label' ] 
#Ru_data.columns =['tweet','label' ] 
Ru_data

In [ ]:
sns.countplot(Ru_data.label)
plt.xlabel('Label')
plt.title('Number of ham and spam messages')

In [ ]:
Ko_data.columns =['tweet','label' ] 
Ko_data

In [ ]:
sns.countplot(Ko_data.label)
plt.xlabel('Label')
plt.title('Number of ham and spam messages')

In [ ]:
data3 = data3[-117382:]

In [ ]:
data1.drop(['lang'],axis=1,inplace=True)

In [ ]:
data2.drop(['id'],axis=1,inplace=True)

In [ ]:

cols = data2.columns.tolist()


cols = cols[-1:] + cols[:-1]

data2 = data2[cols]

data2

In [ ]:
data.columns =['tweet','label' ] 
data1.columns =['tweet','label' ] 



In [ ]:
data3.columns =['label','tweet' ]

In [ ]:
cols = data3.columns.tolist()


cols = cols[-1:] + cols[:-1]

data3 = data3[cols]

data2
data3

In [ ]:
#df= ([data]+[data1]+[data2]+[data3])
#df

In [ ]:
#[data.label]+ [data1.label]+[data2.label]+[data3.label]

In [ ]:
#df = [data.label]+[data1.label]+[data2.label]+[data3.label] 

How are tweets spread among these 2 classes?

In [ ]:
#df.label.split

In [ ]:
#import numpy as np
#print("Hatred labeled: {}\nNon-hatred labeled: {}".format(
#    (data.label+data1.label+data2.label+data3.label  == 1).sum(),
#    (data.label+data1.label+data2.label+data3.label == 0).sum()
#))


In [ ]:
sns.countplot(data.label)
plt.xlabel('Label')
plt.title('Number of ham and spam messages')
#german DS

In [ ]:
sns.countplot(data1.label)
plt.xlabel('Label')
plt.title('Number of ham and spam messages')
#multi language

In [ ]:
sns.countplot(data2.label)
plt.xlabel('Label')
plt.title('Number of ham and spam messages')
# english DS

In [ ]:
sns.countplot(data3.label)
plt.xlabel('Label')
plt.title('Number of ham and spam messages')
# french DS

In [ ]:
def eval_fun(labels, preds):
    labels = label.split(' ')
    preds = tweet.split(' ')
    rr = (np.intersect1d(label, tweet))
    precision = np.float(len(rr)) / len(tweet)
    recall = np.float(len(rr)) / len(label)
    try:
        f1 = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        return (precision, recall, 0.0)
    return (precision, recall, f1)
print(1)

## Extracting features

#### Hashtags and mentions
We'll extract hashtags for each tweet as an extra column to explore them later.   
For user mentions, all of the usernames have been replaced with `'user'` so we can't get any data from it, we'll just remove mentions and keep the number of mentions in each tweet as an extra features for that tweet.  

In [ ]:
# data preprocess
hashtags = data['tweet'].str.extractall('#(?P<hashtag>[a-zA-Z0-9_]+)').reset_index().groupby('level_0').agg(lambda x: ' '.join(x.values))
data.loc[:, 'hashtags'] = hashtags['hashtag']
data['hashtags'].fillna('', inplace=True)

data.loc[:, 'mentions'] = data['tweet'].str.count('@[a-zA-Z0-9_]+')

data.tweet = data.tweet.str.replace('@[a-zA-Z0-9_]+', '')




In [ ]:
# data preprocess
hashtags = Ru_data['tweet'].str.extractall('#(?P<hashtag>[a-zA-Z0-9_]+)').reset_index().groupby('level_0').agg(lambda x: ' '.join(x.values))
Ru_data.loc[:, 'hashtags'] = hashtags['hashtag']
Ru_data['hashtags'].fillna('', inplace=True)

Ru_data.loc[:, 'mentions'] = Ru_data['tweet'].str.count('@[a-zA-Z0-9_]+')

Ru_data.tweet = Ru_data.tweet.str.replace('@[a-zA-Z0-9_]+', '')


In [ ]:
# data preprocess
hashtags = Ko_data['tweet'].str.extractall('#(?P<hashtag>[a-zA-Z0-9_]+)').reset_index().groupby('level_0').agg(lambda x: ' '.join(x.values))
Ko_data.loc[:, 'hashtags'] = hashtags['hashtag']
Ko_data['hashtags'].fillna('', inplace=True)

Ko_data.loc[:, 'mentions'] = Ko_data['tweet'].str.count('@[a-zA-Z0-9_]+')

Ko_data.tweet = Ko_data.tweet.str.replace('@[a-zA-Z0-9_]+', '')


In [ ]:
# data1 preprocess
hashtags1 = data1['tweet'].str.extractall('#(?P<hashtag>[a-zA-Z0-9_]+)').reset_index().groupby('level_0').agg(lambda x: ' '.join(x.values))
data1.loc[:, 'hashtags'] = hashtags1['hashtag']
data1['hashtags'].fillna('', inplace=True)

data1.loc[:, 'mentions'] = data1['tweet'].str.count('@[a-zA-Z0-9_]+')

data.tweet = data.tweet.str.replace('@[a-zA-Z0-9_]+', '')

In [ ]:
# data2 preprocess
hashtags2 = data2['tweet'].str.extractall('#(?P<hashtag>[a-zA-Z0-9_]+)').reset_index().groupby('level_0').agg(lambda x: ' '.join(x.values))
data2.loc[:, 'hashtags'] = hashtags2['hashtag']
data2['hashtags'].fillna('', inplace=True)

data2.loc[:, 'mentions'] = data2['tweet'].str.count('@[a-zA-Z0-9_]+')

data2.tweet = data2.tweet.str.replace('@[a-zA-Z0-9_]+', '')


In [ ]:

# data3 preprocess
hashtags3 = data3['tweet'].str.extractall('#(?P<hashtag>[a-zA-Z0-9_]+)').reset_index().groupby('level_0').agg(lambda x: ' '.join(x.values))
data3.loc[:, 'hashtags'] = hashtags3['hashtag']
data3['hashtags'].fillna('', inplace=True)

data3.loc[:, 'mentions'] = data3['tweet'].str.count('@[a-zA-Z0-9_]+')

data3.tweet = data3.tweet.str.replace('@[a-zA-Z0-9_]+', '')

#### Removing anything but the words
Now we'll remove anything but the words (punctuations, numbers, etc). Note that this time we'll replace them with a blank space since it might be a `_` or `-` or a punctuation with no space from the next word and we don't want the words to join together.  

In [ ]:
data.tweet = data.tweet.str.replace('[^a-zA-Z]', ' ')
Ru_data.tweet = Ru_data.tweet.str.replace('[^a-zA-Z]', ' ')
Ko_data.tweet = Ko_data.tweet.str.replace('[^a-zA-Z]', ' ')


#### Lemmatization
We lemmatize tweets' words as we have the sentences and we can tag part of speeches, and will stem hashtags.  

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, FreqDist, word_tokenize

stemmerG = SnowballStemmer('german')
stemmerF = SnowballStemmer('french')
stemmerE = SnowballStemmer('english')
lemmer = WordNetLemmatizer() 

part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

def convert_tag(penn_tag):
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        return 'n'


def tag_and_lem(element):
    sent = pos_tag(word_tokenize(element))
    return ' '.join([lemmer.lemmatize(sent[k][0], convert_tag(sent[k][1][0]))
                    for k in range(len(sent))])
    

data.loc[:, 'tweet'] = data['tweet'].apply(lambda x: tag_and_lem(x))
data.loc[:, 'hashtags'] = data['hashtags'].apply(lambda x: ' '.join([stemmerG.stem(word) for word in x.split()]))

data1.loc[:, 'tweet'] = data1['tweet'].apply(lambda x: tag_and_lem(x))
data1.loc[:, 'hashtags'] = data1['hashtags'].apply(lambda x: ' '.join(
    [stemmerG.stem(word) for word in x.split()]+
    [stemmerF.stem(word) for word in x.split()]+
    [stemmerE.stem(word) for word in x.split()]))

data2.loc[:, 'tweet'] = data2['tweet'].apply(lambda x: tag_and_lem(x))
data2.loc[:, 'hashtags'] = data2['hashtags'].apply(lambda x: ' '.join([stemmerE.stem(word) for word in x.split()]))

data3.loc[:, 'tweet'] = data3['tweet'].apply(lambda x: tag_and_lem(x))
data3.loc[:, 'hashtags'] = data3['hashtags'].apply(lambda x: ' '.join([stemmerF.stem(word) for word in x.split()]))

In [ ]:
#pip install konlpy

In [ ]:
#from konlpy.tag import Okt
#from konlpy.utils import pprint
#from konlpy.corpus import kobill

In [ ]:
#from konlpy.tag import Twitter; t = Twitter()
#tokens_ko = t.kobill(Ko_data)

In [ ]:
#okt = Okt()

In [ ]:
#Ko_data

In [ ]:
#pprint(okt.pos(Ko_data,norm=True, stem=True)

In [ ]:

from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, FreqDist, word_tokenize

stemmerG = SnowballStemmer('german')
stemmerF = SnowballStemmer('french')
stemmerE = SnowballStemmer('english')

stemmerRu = SnowballStemmer('russian')

#stemmerKo = SnowballStemmer('Korean')
lemmer = WordNetLemmatizer() 

part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

def convert_tag(penn_tag):
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        return 'n'


def tag_and_lem(element):
    sent = pos_tag(word_tokenize(element))
    return ' '.join([lemmer.lemmatize(sent[k][0], convert_tag(sent[k][1][0]))
                    for k in range(len(sent))])
    

data.loc[:, 'tweet'] = data['tweet'].apply(lambda x: tag_and_lem(x))
data.loc[:, 'hashtags'] = data['hashtags'].apply(lambda x: ' '.join([stemmerG.stem(word) for word in x.split()]))

data1.loc[:, 'tweet'] = data1['tweet'].apply(lambda x: tag_and_lem(x))
data1.loc[:, 'hashtags'] = data1['hashtags'].apply(lambda x: ' '.join([stemmerE.stem(word) for word in x.split()]))

#data1.loc[:, 'tweet'] = data1['tweet'].apply(lambda x: tag_and_lem(x))
#data1.loc[:, 'hashtags'] = data1['hashtags'].apply(lambda x: ' '.join([stemmerE.stem(word) for word in x.split()]))

#data1.loc[:, 'tweet'] = data1['tweet'].apply(lambda x: tag_and_lem(x))
#data1.loc[:, 'hashtags'] = data1['hashtags'].apply(lambda x: ' '.join([stemmerF.stem(word) for word in x.split()]))

#data.loc[:, 'tweet'] = data1['tweet'].apply(lambda x: tag_and_lem(x))
#data.loc[:, 'hashtags'] = data1['hashtags'].apply(lambda x: ' '.join([stemmerG.stem(word) for word in x.split()]))





data2.loc[:, 'tweet'] = data2['tweet'].apply(lambda x: tag_and_lem(x))
data2.loc[:, 'hashtags'] = data2['hashtags'].apply(lambda x: ' '.join([stemmerE.stem(word) for word in x.split()]))

data3.loc[:, 'tweet'] = data3['tweet'].apply(lambda x: tag_and_lem(x))
data3.loc[:, 'hashtags'] = data3['hashtags'].apply(lambda x: ' '.join([stemmerF.stem(word) for word in x.split()]))

#Ru_data.loc[:, 'tweet'] =  Ru_data['tweet'].apply(lambda x: tag_and_lem(x))
#Ru_data.loc[:, 'hashtags'] = Ru_data['hashtags'].apply(lambda x: ' '.join([stemmerRu.stem(word) for word in x.split()]))

#Ko_data.loc[:, 'tweet'] = Ko_data['tweet'].apply(lambda x: tag_and_lem(x))
#Ko_data.loc[:, 'hashtags'] = Ko_data['hashtags'].apply(lambda x: ' '.join([stemmerKo.stem(word) for word in x.split()]))


bild model cnn

In [ ]:
texts = np.array(data1['tweet'])
labels = np.array(data1['label'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

X = data1.tweet
y = data1.label
le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.042)

In [ ]:
X_test.shape

In [ ]:
MAX_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
# the percentage of train test split to be applied
VALIDATION_SPLIT = 0.20
# the dimension of vectors to be used
EMBEDDING_DIM = 100
# filter sizes of the different conv layers 
filter_sizes = [3,4,5]
num_filters = 512
embedding_dim = 100
# dropout probability
drop = 0.5
batch_size = 30

In [ ]:
import string
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_words = 10000
max_len = 1000
samples = texts.shape[0]
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
word_index = tokenizer.word_index
print(data1.shape)
print(labels.shape)

In [ ]:
X_train.shape[0]

In [ ]:
X_test.shape

In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping

import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from keras import backend as K

from keras.engine import Layer, InputSpec, InputLayer

from keras.models import Model, Sequential

from keras.layers import Dropout, Embedding, concatenate
from keras.layers import Conv1D, MaxPool1D, Conv2D, MaxPool2D, ZeroPadding1D
from keras.layers import Dense, Input, Flatten, BatchNormalization
from keras.layers import Concatenate, Dot, merge, Multiply, RepeatVector
from keras.layers import Bidirectional, TimeDistributed
from keras.layers import SimpleRNN, LSTM, GRU, Lambda

from keras.layers.core import Reshape, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard
from keras.constraints import maxnorm
from keras.regularizers import l2

In [ ]:
input_dim= X_train.shape[0]

In [ ]:
#from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 

from keras.models import Sequential
#from keras.layers import Dense, Embedding, SimpleRNN, LSTM, GRU, Bidirectional
model_1 = Sequential([(Embedding(1+len(word_index), 16)),
    ZeroPadding1D((49,49)),
    Conv1D(64, 50, padding="same"),
    MaxPooling1D(2),
    Activation("relu"),
    ZeroPadding1D((24,24)),
   # GlobalMaxPooling1D(),
    Dropout(0.5),
    Conv1D(64, 25, padding="same"),
    Dense(10, input_dim=input_dim, activation='sigmoid'),
    Flatten(),
    #Dense(1, activation='sigmoid')                  
    
                      
])

In [ ]:
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_1.summary()

In [ ]:
history = model_1.fit(sequences_matrix,Y_train,batch_size=64,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0002)], verbose=1)

In [ ]:
#max_words = 10000
#max_len = 1000

#samples = texts.shape[0]

#tokenizer = Tokenizer(num_words=max_words)
#tokenizer.fit_on_texts(X_train)
#sequences = tokenizer.texts_to_sequences(X_train)
#sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
#word_index = tokenizer.word_index

test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

accr = model_1.evaluate(test_sequences_matrix,Y_test)
#print(data1.shape)
#print(labels.shape)

In [ ]:
Y_test = model_1.predict(test_sequences_matrix )

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
def cnfmatrix(Y_test,results):
    fp = 0.0
    fn = 0.0
    tp = 0.0
    tn = 0.0
    t = 0.0
    n = 0.0
    results.shape
    for i in range(results.shape[0]):
        if Y_test[i]==1 and results[i]==1:
            tp+=1
            t+=1
        elif Y_test[i]==1 and results[i]==0:
            fn+=1
            t+=1
        elif Y_test[i]==0 and results[i]==1:
            fp+=1
            n+=1
        elif Y_test[i]==0 and results[i]==0:
            tn+=1
            n+=1
    print(tp/results.shape[0],fp/results.shape[0])
    print(fn/results.shape[0],tn/results.shape[0])
    Precision  = tp/(tp+fp)
    Recall = tp/(tp+fn)
    print("Precision: ",Precision,"Recall: ",Recall)
    f1_score = (2*Precision*Recall)/(Precision+Recall)
    print("f1score: ",f1score)
    print("accuracy: ",(tp+tn)/results.shape[0])
    print("hate_acc: ", (tp)/t)
    print("non_hate_acc: ", (tn)/n)

In [ ]:
#inputs = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
#embedding = embedding_layer(inputs)

#print(embedding.shape)
#reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(embedding)
#print(reshape.shape)

#conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
#conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
#conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

#maxpool_0 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
#maxpool_1 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
#maxpool_2 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

#concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
#flatten = Flatten()(concatenated_tensor)
#dropout = Dropout(drop)(flatten)
#output = Dense(units=20, activation='softmax')(dropout)

# this creates a model that includes
#model = Model(inputs=inputs, outputs=output)

#checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
#adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

#model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, LSTM, GRU, Bidirectional
#model = Sequential()
#model.add(Embedding(1+len(word_index), 16))
##model.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.5, return_sequences=True)))
#model.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.5)))
#model.add(Dense(1, activation='sigmoid'))
#model.summary()




In [ ]:
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
#history = model.fit(sequences_matrix,Y_train,batch_size=512,epochs=10,
#          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)], verbose=1)

In [ ]:
texts = np.array(X_test)


In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt  # doctest: +SKIP
from sklearn import datasets, model_selection, svm
from numpy import interp

def plot_roc(y_true, y_probas, title='ROC Curves',plot_micro=True, plot_macro=True, classes_to_plot=None,
                   ax=None, figsize=None, cmap='nipy_spectral',
                   title_fontsize="large", text_fontsize="medium"):
    y_true = np.array(y_true)
    y_probas = np.array(y_probas)

    classes = np.unique(y_true)
    probas = y_probas

    if classes_to_plot is None:
        classes_to_plot = classes

    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=figsize)

    ax.set_title(title, fontsize=title_fontsize)

    fpr_dict = dict()
    tpr_dict = dict()

    indices_to_plot = np.in1d(classes, classes_to_plot)
    for i, to_plot in enumerate(indices_to_plot):
        fpr_dict[i], tpr_dict[i], _ = roc_curve(y_true, probas[:, i],pos_label=classes[i])
        if to_plot:
            roc_auc = auc(fpr_dict[i], tpr_dict[i])
            color = plt.cm.get_cmap(cmap)(float(i) / len(classes))
            ax.plot(fpr_dict[i], tpr_dict[i], lw=2, color=color,
                    label='ROC curve of class {0} (area = {1:0.2f})'
                          ''.format(classes[i], roc_auc))

    if plot_micro:
        binarized_y_true = label_binarize(y_true, classes=classes)
        if len(classes) == 2:
            binarized_y_true = np.hstack(
                (1 - binarized_y_true, binarized_y_true))
        fpr, tpr, _ = roc_curve(binarized_y_true.ravel(), probas.ravel())
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr,
                label='micro-average ROC curve '
                                      '(area = {0:0.2f})'.format(roc_auc),
                color='deeppink', linestyle=':', linewidth=4)

    if plot_macro:
        # Compute macro-average ROC curve and ROC area
        # First aggregate all false positive rates
        all_fpr = np.unique(np.concatenate([fpr_dict[x] for x in range(len(classes))]))

        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(len(classes)):
            mean_tpr += interp(all_fpr, fpr_dict[i], tpr_dict[i])

        # Finally average it and compute AUC
        mean_tpr /= len(classes)
        roc_auc = auc(all_fpr, mean_tpr)

        ax.plot(all_fpr, mean_tpr,
                label='macro-average ROC curve '
                      '(area = {0:0.2f})'.format(roc_auc),
                color='navy', linestyle=':', linewidth=4)

    ax.plot([0, 1], [0, 1], 'k--', lw=2)
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontsize=text_fontsize)
    ax.set_ylabel('True Positive Rate', fontsize=text_fontsize)
    ax.tick_params(labelsize=text_fontsize)
    ax.legend(loc='lower right', fontsize=text_fontsize)
    return ax


# # **English dataset**

1- define the label and text
E = English dataset
ET= English test set


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

E = data2.tweet
ET = data2.label                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
le = LabelEncoder()
ET = le.fit_transform(ET)
ET = ET.reshape(-1,1)

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train,X_test,Y_train,Y_test = train_test_split(E,ET,test_size=0.30)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)


In [ ]:
from sklearn.model_selection import train_test_split
Etrain_train,Etrain_test,ETtest_train,ETtest_test = train_test_split(E,ET,test_size=0.15425)

In [ ]:
print(ETtest_test.shape)

In [ ]:
#ET= [:5000]
#ET = data2[:4931]
#ET = labels[:4931]

In [ ]:
#accr = model_1.evaluate(test_sequences_matrix,Y_test)

accr = model_1.evaluate(test_sequences_matrix,ETtest_test)


In [ ]:
ETtest_test1= model_1.predict(test_sequences_matrix)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
def cnfmatrix(Y_test,results):
    fp = 0.0
    fn = 0.0
    tp = 0.0
    tn = 0.0
    t = 0.0
    n = 0.0
    results.shape
    for i in range(results.shape[0]):
        if Y_test[i]==1 and results[i]==1:
            tp+=1
            t+=1
        elif Y_test[i]==1 and results[i]==0:
            fn+=1
            t+=1
        elif Y_test[i]==0 and results[i]==1:
            fp+=1
            n+=1
        elif Y_test[i]==0 and results[i]==0:
            tn+=1
            n+=1
    print(tp/results.shape[0],fp/results.shape[0])
    print(fn/results.shape[0],tn/results.shape[0])
    Precision  = tp/(tp+fp)
    Recall = tp/(tp+fn)
    print("Precision: ",Precision,"Recall: ",Recall)
    f1_score = (2*Precision*Recall)/(Precision+Recall)
    print("f1score: ",f1score)
    print("accuracy: ",(tp+tn)/results.shape[0])
    print("hate_acc: ", (tp)/t)
    print("non_hate_acc: ", (tn)/n)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# **French dataset**
F = French dataset
FT= French test set

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

F = data3.tweet
FT = data3.label
le = LabelEncoder()
FT = le.fit_transform(FT)
FT = FT.reshape(-1,1)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)


4930/2= 2465
170810-2465=  168345
170810+2465=   173275

In [ ]:
from sklearn.model_selection import train_test_split
Ftrain_train,Ftrain_test,FTtest_train,FTtest_test = train_test_split(F,FT,test_size=0.042)

In [ ]:
FTtest_test.shape

# 4931 

In [ ]:
#accr = model_1.evaluate(test_sequences_matrix,Y_test)
#FT_test = model_1.predict(test_sequences_matrix)
accr = model_1.evaluate(test_sequences_matrix,FTtest_test)



FTtest_test1 = model_1.predict(test_sequences_matrix)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
#FT_test = model_1.predict(test_sequences_matrix )

In [ ]:
def cnfmatrix(Y_test,results):
    fp = 0.0
    fn = 0.0
    tp = 0.0
    tn = 0.0
    t = 0.0
    n = 0.0
    results.shape
    for i in range(results.shape[0]):
        if Y_test[i]==1 and results[i]==1:
            tp+=1
            t+=1
        elif Y_test[i]==1 and results[i]==0:
            fn+=1
            t+=1
        elif Y_test[i]==0 and results[i]==1:
            fp+=1
            n+=1
        elif Y_test[i]==0 and results[i]==0:
            tn+=1
            n+=1
    print(tp/results.shape[0],fp/results.shape[0])
    print(fn/results.shape[0],tn/results.shape[0])
    Precision  = tp/(tp+fp)
    Recall = tp/(tp+fn)
    print("Precision: ",Precision,"Recall: ",Recall)
    f1_score = (2*Precision*Recall)/(Precision+Recall)
    print("f1score: ",f1score)
    print("accuracy: ",(tp+tn)/results.shape[0])
    print("hate_acc: ", (tp)/t)
    print("non_hate_acc: ", (tn)/n)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# # **German Dataset**
G = German dataset
GT= German test set

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

G = data.tweet
GT = data.label
le = LabelEncoder()
GT = le.fit_transform(GT)
GT = GT.reshape(-1,1)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
#test_sequences = tokenizer.texts_to_sequences(Gtrain_train)
#test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
from sklearn.model_selection import train_test_split
Gtrain_train,Gtrain_test,GTtest_train,GTtest_test = train_test_split(G,GT,test_size=0.9845)

In [ ]:
#GT = data[:4931]
#GT = labels[:4931]

In [ ]:
#accr = model_1.evaluate(test_sequences_matrix,Y_test)
#FT_test = model_1.predict(test_sequences_matrix)
accr = model_1.evaluate(test_sequences_matrix,GTtest_test)


GTtest_test1 = model_1.predict(test_sequences_matrix)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

**other datasets** \n
these datasets are not trained on our models
we would like to include other lnaguages 


# ROC/ AUC

In [ ]:
#from sklearn.metrics import roc_curve
#from sklearn.metrics import auc
#GTtest_test1 = model_1.predict(test_sequences_matrix).ravel()




In [ ]:
#GTtest_test1 = model_1.predict(test_sequences_matrix)

In [ ]:
#fpr_GT, tpr_GT, thresholds_keras = roc_curve(GTtest_test, GTtest_test1)
#auc_GT = auc(fpr_GT, tpr_GT)

In [ ]:
#plt.figure(1)
#plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(fpr_GT, tpr_GT, label='test (GT CNN = {:.4f})'.format(auc_GT))
#plt.plot(fpr_keras1, tpr_keras1, label='train (area = {:.3f})'.format(auc_keras1))
#plt.plot(fpr_keras2, tpr_keras2, label='val (area = {:.3f})'.format(auc_keras2))
#plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
#plt.xlabel('False positive rate')
#plt.ylabel('True positive rate')
#plt.title('ROC curve')
#plt.legend(loc='best')
#plt.show()

# > > RNN model

In [ ]:
A = data1.tweet
B = data1.label
le = LabelEncoder()
B = le.fit_transform(B)
B = B.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split
A_train,A_test,B_train,B_test = train_test_split(X,y,test_size=0.042)

In [ ]:
max_words = 10000
max_len = 1000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(A_train)
sequences = tok.texts_to_sequences(A_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(64,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(sequences_matrix,B_train,batch_size=128,epochs=10,
          validation_split=0.02,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
#test_sequences = tokenizer.texts_to_sequences(X_test)
#test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(A_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:

accr = model.evaluate(test_sequences_matrix,B_test)
#accr = model.evaluate(test_sequences_matrix,B_test)
B_test1 = model.predict(test_sequences_matrix)

In [ ]:


print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
def cnfmatrix(B_test,results):
    fp = 0.0
    fn = 0.0
    tp = 0.0
    tn = 0.0
    t = 0.0
    n = 0.0
    results.shape
    for i in range(results.shape[0]):
        if Y_test[i]==1 and results[i]==1:
            tp+=1
            t+=1
        elif Y_test[i]==1 and results[i]==0:
            fn+=1
            t+=1
        elif Y_test[i]==0 and results[i]==1:
            fp+=1
            n+=1
        elif Y_test[i]==0 and results[i]==0:
            tn+=1
            n+=1
    print(tp/results.shape[0],fp/results.shape[0])
    print(fn/results.shape[0],tn/results.shape[0])
    Precision  = tp/(tp+fp)
    Recall = tp/(tp+fn)
    print("Precision: ",Precision,"Recall: ",Recall)
    f1_score = (2*Precision*Recall)/(Precision+Recall)
    print("f1score: ",f1score)
    print("accuracy: ",(tp+tn)/results.shape[0])
    print("hate_acc: ", (tp)/t)
    print("non_hate_acc: ", (tn)/n)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# English dataset

ER= English rnn train
ETR= English test RNN

In [ ]:
ER = data2.tweet
ETR = data2.label
le = LabelEncoder()
ETR = le.fit_transform(ETR)
ETR = ETR.reshape(-1,1)

In [ ]:
#test_sequences = tokenizer.texts_to_sequences(X_test)
#test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(A_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
from sklearn.model_selection import train_test_split
Etrain_train,Etrain_test,ETtest_train,ETtest_test = train_test_split(ER,ETR,test_size=0.15425)

In [ ]:
#ETR = data2[:4931]
#ETR = labels[:4931]

In [ ]:
accr = model.evaluate(test_sequences_matrix,ETtest_test)

#accr = model.evaluate(test_sequences_matrix,ETR)
ETtest_test1 = model.predict(test_sequences_matrix)

print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# French dataset 

FR= french rnn train
FTR= french test RNN

In [ ]:
FR = data3.tweet
FTR = data3.label
le = LabelEncoder()
FTR = le.fit_transform(FTR)
FTR = FTR.reshape(-1,1)

In [ ]:
#test_sequences = tokenizer.texts_to_sequences(X_test)
#test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
test_sequences = tok.texts_to_sequences(A_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:

F_train,F_test,FT_train,FT_test = train_test_split(FR,FTR,test_size=0.042)

In [ ]:
#FTR = data3[168345:173275]
#FTR = labels[168345:173275]

In [ ]:
F_test.shape

In [ ]:
accr = model.evaluate(test_sequences_matrix,FT_test)
#accr = model.evaluate(test_sequences_matrix,FT_test)

print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
FT_test1 = model.predict(test_sequences_matrix)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# Germany DATAset

GR= germany rnn train
GTR= Germany test RNN

In [ ]:
GR = data.tweet
GTR = data.label
le = LabelEncoder()
GTR = le.fit_transform(GTR)
GTR = GTR.reshape(-1,1)

In [ ]:
#test_sequences = tokenizer.texts_to_sequences(X_test)
#test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
test_sequences = tok.texts_to_sequences(A_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
from sklearn.model_selection import train_test_split
Gtrain_train,Gtrain_test,GTtest_train,GTtest_test = train_test_split(GR,GTR,test_size=0.9845)

In [ ]:
#GTR = data[:4931]
#GTR = labels[:4931]

In [ ]:

accr = model.evaluate(test_sequences_matrix,GTtest_test)

print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
GTtest_test1 = model.predict(test_sequences_matrix)

In [ ]:
def cnfmatrix(B_test,results):
    fp = 0.0
    fn = 0.0
    tp = 0.0
    tn = 0.0
    t = 0.0
    n = 0.0
    results.shape
    for i in range(results.shape[0]):
        if Y_test[i]==1 and results[i]==1:
            tp+=1
            t+=1
        elif Y_test[i]==1 and results[i]==0:
            fn+=1
            t+=1
        elif Y_test[i]==0 and results[i]==1:
            fp+=1
            n+=1
        elif Y_test[i]==0 and results[i]==0:
            tn+=1
            n+=1
    print(tp/results.shape[0],fp/results.shape[0])
    print(fn/results.shape[0],tn/results.shape[0])
    Precision  = tp/(tp+fp)
    Recall = tp/(tp+fn)
    print("Precision: ",Precision,"Recall: ",Recall)
    f1_score = (2*Precision*Recall)/(Precision+Recall)
    print("f1score: ",f1score)
    print("accuracy: ",(tp+tn)/results.shape[0])
    print("hate_acc: ", (tp)/t)
    print("non_hate_acc: ", (tn)/n)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# End of testing and evaluating

In [ ]:
#prediction function 
from keras.models import Sequential
predictions = model_1.predict(B_test)





In [ ]:
#predictions = model.predict(ET)
#predictions1 = model.predict(FT_test)
#predictions2 = model.predict(GT)

In [ ]:

import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt  # doctest: +SKIP
from sklearn import datasets, model_selection, svm
from numpy import interp

In [ ]:

def plot_roc(y_true, y_probas, title='ROC Curves',plot_micro=True, plot_macro=True, classes_to_plot=None,
                   ax=None, figsize=None, cmap='nipy_spectral',
                   title_fontsize="large", text_fontsize="medium"):
    y_true = np.array(y_true)
    y_probas = np.array(y_probas)

    classes = np.unique(y_true)
    probas = y_probas

    if classes_to_plot is None:
        classes_to_plot = classes

    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=figsize)

    ax.set_title(title, fontsize=title_fontsize)

    fpr_dict = dict()
    tpr_dict = dict()

    indices_to_plot = np.in1d(classes, classes_to_plot)
    for i, to_plot in enumerate(indices_to_plot):
        fpr_dict[i], tpr_dict[i], _ = roc_curve(y_true, probas[:, i],pos_label=classes[i])
        if to_plot:
            roc_auc = auc(fpr_dict[i], tpr_dict[i])
            color = plt.cm.get_cmap(cmap)(float(i) / len(classes))
            ax.plot(fpr_dict[i], tpr_dict[i], lw=2, color=color,
                    label='ROC curve of class {0} (area = {1:0.2f})'
                          ''.format(classes[i], roc_auc))

    if plot_micro:
        binarized_y_true = label_binarize(y_true, classes=classes)
        if len(classes) == 2:
            binarized_y_true = np.hstack(
                (1 - binarized_y_true, binarized_y_true))
        fpr, tpr, _ = roc_curve(binarized_y_true.ravel(), probas.ravel())
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr,
                label='micro-average ROC curve '
                                      '(area = {0:0.2f})'.format(roc_auc),
                color='deeppink', linestyle=':', linewidth=4)

    if plot_macro:
        # Compute macro-average ROC curve and ROC area
        # First aggregate all false positive rates
        all_fpr = np.unique(np.concatenate([fpr_dict[x] for x in range(len(classes))]))

        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(len(classes)):
            mean_tpr += interp(all_fpr, fpr_dict[i], tpr_dict[i])

        # Finally average it and compute AUC
        mean_tpr /= len(classes)
        roc_auc = auc(all_fpr, mean_tpr)

        ax.plot(all_fpr, mean_tpr,
                label='macro-average ROC curve '
                      '(area = {0:0.2f})'.format(roc_auc),
                color='navy', linestyle=':', linewidth=4)

    ax.plot([0, 1], [0, 1], 'k--', lw=2)
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontsize=text_fontsize)
    ax.set_ylabel('True Positive Rate', fontsize=text_fontsize)
    ax.tick_params(labelsize=text_fontsize)
    ax.legend(loc='lower right', fontsize=text_fontsize)
    return ax


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

X = data.tweet
y = data.label
le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.30)

In [ ]:
X, y = datasets.make_classification(random_state=0)
X_train, X_test, y_train, y_test = model_selection.train_test_split(            X, y, random_state=0)
clf = svm.SVC(random_state=0,probability=True)
clf.fit(X_train, y_train)
svm.SVC(random_state=0)

In [ ]:
y_probas = clf.predict_proba(X_test)
y_true = y_test

plot_roc(y_true, y_probas)

In [ ]:
from sklearn.metrics import roc_curve
y_pred_keras =  model_1.predict(y_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Supervised transformation based on random forests
rf = RandomForestClassifier(max_depth=3, n_estimators=10)
rf.fit(X_train, Y_train)

y_pred_rf = rf.predict_proba(x_test)[:, 1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_pred_rf)
auc_rf = auc(fpr_rf, tpr_rf)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.


In [ ]:
results = []
for prediction in predictions:
    if prediction < 0.5:
        results.append(0)
    else:
        results.append(1)
        
results = np.array(results)

In [ ]:
cnfmatrix(y_test, results)


In [ ]:
def test(model):
    f1_score = lambda precision, recall: 2 * ((precision * recall) / (precision + recall))
    nexamples, recall, precision = model.test('fasttext.test')
    print (f'recall: {recall}' )
    print (f'precision: {precision}')
    print (f'f1 score: {f1_score(precision,recall)}')
    print (f'number of examples: {nexamples}')
print(1)

In [ ]:
!pip install plot-metric

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=1000, n_classes=2, weights=[1,1], random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=2)

In [ ]:
import scipy
from scipy.io import arff
data = (data)
from sklearn.datasets import make_multilabel_classification

# this will generate a random multi-label dataset
X, y = make_multilabel_classification(sparse = True, n_labels = 20,
return_indicator = 'sparse', allow_unlabeled = False)

# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50, random_state=23)
model = clf.fit(X_train, y_train)

# Use predict_proba to predict probability of the class
y_pred = clf.predict_proba(X_test)[:,1]

In [ ]:
import matplotlib.pyplot as plt  
from sklearn import datasets, metrics, model_selection, svm

In [ ]:
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])#
#history = model.fit(x_train,y_train,epochs=20,batch_size=512,validation_data=(x_val,y_val))

#### Frequent words
Now let's see what words and hashtags are the most frequenst in hate tweets and in total.

In [ ]:
##from wordcloud import WordCloud, STOPWORDS
##stopwords = STOPWORDS.add('amp')
##traindata = ['tweet', 'label']
##all_words = ' '.join('tweet', 'label')
##hatred_words = ' '.join(data[data.label == 1].tweet.values)

##plt.figure(figsize=(16, 8))

##cloud1 = WordCloud(width=400, height=400, background_color='white', stopwords=stopwords).generate(all_words)
##plt.subplot(121)
##plt.imshow(cloud1, interpolation="bilinear")
#plt.axis("off")
#plt.title('All tweets', size=20)

##cloud2 = WordCloud(width=400, height=400, background_color='white', stopwords=stopwords).generate(hatred_words)
##plt.subplot(122)
##plt.imshow(cloud2, interpolation="bilinear")
##plt.axis("off")
##plt.title('Hatred tweets', size=20)
##plt.show()

#### Hashtags
Now let's see which hashtags are used the most in hatred tweets and in total.

In [ ]:
##all_hashtags = FreqDist(list(' '.join(data.hashtags.values).split())).most_common(10)
##hatred_hashtags = FreqDist(list(' '.join(data[data.label==1].hashtags.values).split())).most_common(10)
##plt.figure(figsize=(14, 6))
##ax = plt.subplot(121)
##pd.DataFrame(all_hashtags, columns=['hashtag', 'Count']).set_index('hashtag').plot.barh(ax=ax, fontsize=12)
##plt.xlabel('# occurrences')
##plt.title('Hashtags in all tweets', size=13)
##ax = plt.subplot(122)
##pd.DataFrame(hatred_hashtags, columns=['hashtag', 'Count']).set_index('hashtag').plot.barh(ax=ax, fontsize=12)
##plt.xlabel('# occurrences')
##plt.ylabel('')
##plt.title('Hashtags in hatred tweets', size=13)
##plt.show()

#### Mentions
Let's see how many mentions are there in total and if they can be of any use

In [ ]:
##print("Number of mentions: {}\nNumber of tweets having a mention: {}\nCorrelation with label: {}".format(
##    data.mentions.sum(),
##    len(data[data.mentions > 0]),
##    np.corrcoef(data.mentions, data.label)[0][1]
##))

There is no meaningful relation between number of mentions and it looks like there is not a correlation either. So we'll remove number of mentions and won't use it as a feature.  

In [ ]:
##data.drop('mentions', axis=1, inplace=True)

#### Tfidf vectorizing
Now we use the frequency of each word in tweets as our features

In [ ]:
##from sklearn.feature_extraction.text import TfidfVectorizer
##from nltk.corpus import stopwords

##vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=10)
##features = vectorizer.fit_transform(data.tweet)

## Classifying
We'll build a SVC and a LogsiticRegression model for classifying our tweets.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score

